নরমাল লাইব্রেরী ইম্পোর্ট।
pandas = এক্সেল শিটের মত কাজ করার জন্য
numpy = array + many mathmatical operation
graphical library: matplotlib & seaborn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Colab ke drive chinanor jonno

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


drive e je folder e data gula rakhsi oita chenanor jonno

In [3]:
cd gdrive/My Drive/Colab Notebooks/Hate Speech Detection

/content/gdrive/My Drive/Colab Notebooks/Hate Speech Detection


dataset ta import korsi porer line ta diye

In [4]:
dataset = pd.read_csv('train.csv')

In [5]:
dataset['speech']

0       #DhoniKeepsTheGlove | WATCH: Sports Minister K...
1       @politico No. We should remember very clearly ...
2       @cricketworldcup Guess who would be the winner...
3       Corbyn is too politically intellectual for #Bo...
4       All the best to #TeamIndia for another swimmin...
                              ...                        
5847    @davidfrum @trueblueusa1 That's cute and all, ...
5848    a recession issa comin' #maga #magamyass #fuck...
5849    #DoctorsFightBack  Will 'The Mad n Irrational ...
5850    #ShiningIndia #educatedindia or more like RUND...
5851    Could this be our new Prime Minister?     #Ric...
Name: speech, Length: 5852, dtype: object

text processing er jonno ei sob library lagse

In [6]:
import nltk
import string
from nltk.corpus import stopwords # stopwords er library
from nltk.stem import WordNetLemmatizer # lemmatizing er library
from sklearn.feature_extraction.text import CountVectorizer # vectorization er library
from sklearn.feature_extraction.text import TfidfTransformer # tfidf er library

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
lemmatizer = WordNetLemmatizer()

In [9]:
def text_process(speech):
    no_punctuation = [c for c in speech if c not in string.punctuation] #remove all the punctuation
    no_punctuation = ''.join(no_punctuation)
    return [lemmatizer.lemmatize(word) for word in no_punctuation.split() if word.lower() not in set(stopwords.words('english'))] # removing all the stopwords + Lemmatizing

In [10]:
bow_transformer = CountVectorizer(analyzer= text_process).fit(dataset['speech']) #vectorization using CountVectorizer

In [11]:
speeches_bow = bow_transformer.transform(dataset['speech'])

In [12]:
tfidftransformer = TfidfTransformer().fit(speeches_bow) #normalization using tf&idf

In [13]:
speeches_tfidf = tfidftransformer.transform(speeches_bow)

In [ ]:
speeches_tfidf.shape

(5852, 24387)

nicher sob gula deep learning er library.

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [15]:
model = Sequential()

In [16]:
model.add(Dense(24387, input_shape = (24387,), activation = 'relu')) # Inputlayer + hidden layer activation function using "RELU"

In [17]:
model.add(Dense(4, activation='softmax')) # Outputlayer + Step function using "softmax"

In [18]:
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics = ['accuracy']) # optimizing using "Adam"

In [19]:
speeches_array = speeches_tfidf.toarray()

In [20]:
y = pd.get_dummies(dataset['label']) #one hot encoding

In [21]:
y

,HATE,NONE,OFFN,PRFN
0,0,1,0,0
1,1,0,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
5847,0,0,0,1
5848,0,1,0,0
5849,0,0,1,0
5850,0,0,1,0


In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(speeches_array, y, test_size=0.20, random_state=42) # train-test split, test size = 20%

In [24]:
model.fit(X_train,y_train,batch_size=100, epochs= 15) # FITTING THE MODEL INTO ARTIFICIAL NEURAL NETWORK

Epoch 1/15
47/47 [==============================] - 7s 153ms/step - loss: 1.0241 - accuracy: 0.6056
Epoch 2/15
47/47 [==============================] - 7s 154ms/step - loss: 0.3182 - accuracy: 0.9069
Epoch 3/15
47/47 [==============================] - 7s 155ms/step - loss: 0.0756 - accuracy: 0.9810
Epoch 4/15
47/47 [==============================] - 7s 157ms/step - loss: 0.0351 - accuracy: 0.9889
Epoch 5/15
47/47 [==============================] - 7s 159ms/step - loss: 0.0249 - accuracy: 0.9930
Epoch 6/15
47/47 [==============================] - 8s 160ms/step - loss: 0.0222 - accuracy: 0.9927
Epoch 7/15
47/47 [==============================] - 8s 161ms/step - loss: 0.0189 - accuracy: 0.9930
Epoch 8/15
47/47 [==============================] - 8s 162ms/step - loss: 0.0146 - accuracy: 0.9947
Epoch 9/15
47/47 [==============================] - 8s 165ms/step - loss: 0.0114 - accuracy: 0.9955
Epoch 10/15
47/47 [==============================] - 8s 166ms/step - loss: 0.0118 - accuracy: 0.9955

In [25]:
predictions = model.predict(X_test) # PREDICTIONS 

In [26]:
pred = np.argmax(predictions, axis = 1)
y_t = y_test.to_numpy()
y_class = np.argmax(y_t, axis = 1)


In [27]:
from sklearn.metrics import classification_report, confusion_matrix

In [28]:
print(classification_report(pred,y_class))

              precision    recall  f1-score   support

           0       0.19      0.28      0.23       147
           1       0.82      0.67      0.74       908
           2       0.15      0.33      0.21        42
           3       0.26      0.46      0.33        74

    accuracy                           0.59      1171
   macro avg       0.36      0.43      0.38      1171
weighted avg       0.68      0.59      0.63      1171



উপরের সব কাজ আগের মত করসি test data র জন্য।

In [ ]:
test_data = pd.read_csv('test.csv')

In [ ]:
bow_transformer1 = CountVectorizer(analyzer= text_process).fit(test_data['speech'])

In [ ]:
speeches_bow1 = bow_transformer.transform(test_data['speech'])

In [ ]:
tfidftransformer1 = TfidfTransformer().fit(speeches_bow1)

In [ ]:
speeches_tfidf1 = tfidftransformer.transform(speeches_bow1)

In [ ]:
speeches_tfidf1 = speeches_tfidf1.toarray()

In [ ]:
predictions1 = model.predict(speeches_tfidf1)

In [ ]:
pred1 = np.argmax(predictions1, axis = 1)

In [ ]:
x1 = pd.DataFrame(test_data['id'], columns=['id'])

In [ ]:
y1 = pd.DataFrame(pred,columns=['label'])

In [ ]:
new_dataframe = pd.concat([x1, y1], axis = 1)

In [ ]:
new_list = list(new_dataframe['label'])
i = 0
for each in new_list:
  if each == 0:
    new_dataframe['label'][i] = 'HATE'
  elif each == 1:
    new_dataframe['label'][i] = 'NONE'
  elif each == 2:
    new_dataframe['label'][i] = 'OFFN'
  elif each == 3:
    new_dataframe['label'][i] = 'PRFN'
  i = i + 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
np.savetxt(r'predict.txt', new_dataframe.values, fmt='%s', delimiter='\t')